**Import**

In [9]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
df = pd.read_csv('/content/drive/MyDrive/EMSE 6574 Project/df_dummied.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head()

,price,year,manufacturer,model,odometer,manu_model,condition_excellent,condition_fair,condition_good,condition_like new,condition_new,condition_salvage,cylinders_12 cylinders,cylinders_3 cylinders,cylinders_4 cylinders,cylinders_5 cylinders,cylinders_6 cylinders,cylinders_8 cylinders,cylinders_UNKNOWN,cylinders_other,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,title_status_clean,title_status_lien,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,transmission_automatic,transmission_manual,transmission_other,drive_UNKNOWN,drive_fwd,drive_rwd,size_compact,size_full-size,size_mid-size,...,model_class_honda accord,model_class_honda civic,model_class_honda cr-v,model_class_honda odyssey,model_class_hyundai,model_class_hyundai sonata,model_class_infiniti,model_class_jaguar,model_class_jeep,model_class_jeep grand cherokee,model_class_jeep wrangler,model_class_kia,model_class_lexus,model_class_lincoln,model_class_mazda,model_class_mercedes-benz,model_class_mercury,model_class_mini,model_class_mitsubishi,model_class_nissan,model_class_nissan altima,model_class_pontiac,model_class_porsche,model_class_ram,model_class_ram 1500,model_class_ram 2500,model_class_rover,model_class_saturn,model_class_subaru,model_class_subaru forester,model_class_subaru outback,model_class_tesla,model_class_toyota,model_class_toyota camry,model_class_toyota corolla,model_class_toyota prius,model_class_toyota rav4,model_class_toyota tacoma,model_class_volkswagen,model_class_volvo
0,33590,2014,gmc,sierra 1500 crew cab slt,57923.0,gmc sierra 1500 crew cab slt,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,22590,2010,chevrolet,silverado 1500,71229.0,chevrolet silverado 1500,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,39590,2020,chevrolet,silverado 1500 crew,19160.0,chevrolet silverado 1500 crew,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,30990,2017,toyota,tundra double cab sr,41124.0,toyota tundra double cab sr,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,15000,2013,ford,f-150 xlt,128000.0,ford f-150 xlt,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
df["odometer"] = np.sqrt(preprocessing.minmax_scale(df["odometer"]))
df.head()

,price,year,manufacturer,model,odometer,manu_model,condition_excellent,condition_fair,condition_good,condition_like new,condition_new,condition_salvage,cylinders_12 cylinders,cylinders_3 cylinders,cylinders_4 cylinders,cylinders_5 cylinders,cylinders_6 cylinders,cylinders_8 cylinders,cylinders_UNKNOWN,cylinders_other,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,title_status_clean,title_status_lien,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,transmission_automatic,transmission_manual,transmission_other,drive_UNKNOWN,drive_fwd,drive_rwd,size_compact,size_full-size,size_mid-size,...,model_class_honda accord,model_class_honda civic,model_class_honda cr-v,model_class_honda odyssey,model_class_hyundai,model_class_hyundai sonata,model_class_infiniti,model_class_jaguar,model_class_jeep,model_class_jeep grand cherokee,model_class_jeep wrangler,model_class_kia,model_class_lexus,model_class_lincoln,model_class_mazda,model_class_mercedes-benz,model_class_mercury,model_class_mini,model_class_mitsubishi,model_class_nissan,model_class_nissan altima,model_class_pontiac,model_class_porsche,model_class_ram,model_class_ram 1500,model_class_ram 2500,model_class_rover,model_class_saturn,model_class_subaru,model_class_subaru forester,model_class_subaru outback,model_class_tesla,model_class_toyota,model_class_toyota camry,model_class_toyota corolla,model_class_toyota prius,model_class_toyota rav4,model_class_toyota tacoma,model_class_volkswagen,model_class_volvo
0,33590,2014,gmc,sierra 1500 crew cab slt,0.380536,gmc sierra 1500 crew cab slt,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,22590,2010,chevrolet,silverado 1500,0.421987,chevrolet silverado 1500,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,39590,2020,chevrolet,silverado 1500 crew,0.218861,chevrolet silverado 1500 crew,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,30990,2017,toyota,tundra double cab sr,0.320640,toyota tundra double cab sr,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,15000,2013,ford,f-150 xlt,0.565686,ford f-150 xlt,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#1. **Random Forest Regression Model**




In [16]:
from sklearn import metrics

In [17]:
def reg_metrics(prediction_model, X_train, X_test, y_train, y_test):
    pred = prediction_model.predict(X_test)
    train_pred = prediction_model.predict(X_train)
    RMSE_train = np.sqrt(metrics.mean_squared_error(y_train, train_pred))
    RMSE_test =  np.sqrt(metrics.mean_squared_error(y_test, pred))
    R2_train = np.round(prediction_model.score(X_train, y_train),4)
    R2_test = np.round(prediction_model.score(X_test, y_test),4)
    print('RMSE on train data:', RMSE_train)
    print('RMSE on test data:', RMSE_test)
    print('R-square on train data:', R2_train)
    print('R-square on test data:', R2_test)
    return (RMSE_train,RMSE_test, R2_train, R2_test)

In [19]:
def reg_metrics_0(prediction_model, X_train, X_test, y_train, y_test):
    pred = prediction_model.predict(X_test)
    train_pred = prediction_model.predict(X_train)
    pred_0 = pred*(pred>0)  # if pred less than 0, let pred=0, that will be more realistic
    train_pred_0 = train_pred*(train_pred>0)
    RMSE_train = np.sqrt(metrics.mean_squared_error(y_train, train_pred_0))
    RMSE_test =  np.sqrt(metrics.mean_squared_error(y_test, pred_0))
    R2_train = np.round(prediction_model.score(X_train, y_train),4)
    R2_test = np.round(prediction_model.score(X_test, y_test),4)
    print('RMSE on train data:', RMSE_train)
    print('RMSE on test data:', RMSE_test)
    print('R-square on train data:', R2_train)
    print('R-square on test data:', R2_test)
    return (RMSE_train,RMSE_test, R2_train, R2_test)

In [20]:
# Create a dataframe to summarize accuracies
prediction_model_list = ['Linear Regression','Log Data Linear Regression','Decision Tree','Random Forest Regression','XGBoost Regression']
acc_cols = ['RMSE_train','RMSE_test','R2_train','R2_test']
accuracy = pd.DataFrame(columns=acc_cols, index=prediction_model_list)

In [13]:
X = df.drop(['price','manufacturer', 'model', 'manu_model'],axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=666)

In [21]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_acc = reg_metrics(rf, X_train, X_test, y_train, y_test)
accuracy.loc['Random Forest Regression'] = rf_acc

RMSE on train data: 2144.9013024210235
RMSE on test data: 5767.273802077912
R-square on train data: 0.9769
R-square on test data: 0.8338


In [22]:
rfprediction = rf.predict(X_test)
rfdf = pd.DataFrame(rfprediction, columns = ['predict price'])
rfdf.head()

,predict price
0,35380.15
1,8186.95
2,52843.54
3,11019.87
4,1479.26




---








---









---



### With the prediction of Random Forest Regression, we want to judge whether the deal is good or bad by comparing the price to the predicted price.


## **Good/Bad Deal**


In [23]:
def Good_Bad_Deal(model_name):
    rfdf = pd.DataFrame(rfprediction, columns = ['predict price'],index=X_test.index)
    df2 = pd.concat([X_test, rfdf], axis=1)
    df3 = df2.loc[df2[model_name] == 1]
    df4 = df3[[model_name,'predict price']]
    df5 = df4.join(df)
    comparison_column = np.where(df5["predict price"] > df5["price"], 'Good Deal', 'Bad Deal')
    df6 = pd.DataFrame(comparison_column, columns=['Good/Bad Deal'], index=df5.index)
    df6 = df5.join(df6)
    df6 = df6[df6['price'] > 1000]
    return df6

In [24]:
Good_Bad_Deal('model_class_kia')

ValueError: ignored

In [ ]:
Good_Bad_Deal('model_class_jeep wrangler')

,model_class_jeep wrangler,predict price,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,Good/Bad Deal
245634,1,35522.05,36683,2017,jeep,wrangler unlimited sport,UNKNOWN,UNKNOWN,gas,50612.0,clean,automatic,4wd,UNKNOWN,SUV,grey,wa,Bad Deal
215405,1,31712.00,31590,2017,jeep,wrangler unlimited sport,good,6 cylinders,gas,30577.0,clean,other,4wd,UNKNOWN,other,UNKNOWN,tn,Good Deal
22052,1,40517.00,39990,2018,jeep,wrangler unlimited all new,good,6 cylinders,other,15688.0,clean,other,4wd,UNKNOWN,other,silver,ca,Good Deal
44404,1,10061.43,10800,2002,jeep,wrangler,UNKNOWN,6 cylinders,gas,158000.0,clean,manual,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,co,Bad Deal
185489,1,20775.28,9500,1995,jeep,wrangler,like new,UNKNOWN,gas,17000.0,clean,automatic,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,oh,Good Deal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128615,1,7138.37,8500,1997,jeep,wrangler,good,4 cylinders,gas,189065.0,clean,automatic,4wd,UNKNOWN,wagon,red,mi,Bad Deal
233021,1,28287.92,39998,2014,jeep,wrangler unlimited rubicon,excellent,6 cylinders,gas,71200.0,clean,manual,4wd,full-size,SUV,black,tx,Bad Deal
240005,1,9571.15,6300,1995,jeep,wrangler,excellent,4 cylinders,gas,204000.0,clean,manual,4wd,UNKNOWN,UNKNOWN,white,va,Good Deal
18109,1,11844.95,12900,1998,jeep,wrangler,excellent,4 cylinders,gas,79300.0,clean,manual,4wd,UNKNOWN,UNKNOWN,white,ca,Bad Deal


## **To see the price is how much cheaper than predicted price and find a Best_Car**


In [ ]:
def Best_Car(model_name):
    df = pd.read_csv('/content/drive/MyDrive/Project_for_EMSE6574/cars_2021_clean(price<104000).csv')
    df.drop('Unnamed: 0',axis=1,inplace=True)
    rfdf = pd.DataFrame(rfprediction, columns = ['predict price'],index=X_test.index)
    df2 = pd.concat([X_test, rfdf], axis=1)
    df3 = df2.loc[df2[model_name] == 1]
    df4 = df3[[model_name,'predict price']]
    df5 = df4.join(df)
    comparison_column = np.where(df5["predict price"] > df5["price"], 'Good Deal', 'Bad Deal')
    df6 = pd.DataFrame(comparison_column, columns=['Good/Bad Deal'], index=df5.index)
    df6 = df5.join(df6)
    df6 = df6[df6['price'] > 1000]
    df7 = pd.DataFrame((df6['predict price'] - df6['price'])/df6['predict price'],columns=['Cheaper Than Predict'])
    df7.style.format({
    'Cheaper Than Predict': '{:,.2%}'.format,
})
    df7 = df7.sort_values(['Cheaper Than Predict'])
    df7['Cheaper Than Predict'] = df7['Cheaper Than Predict'].astype(float).map("{:.2%}".format)
    df = pd.read_csv('/content/drive/MyDrive/Project_for_EMSE6574/cars_2021_clean(price<104000).csv')
    df.drop('Unnamed: 0',axis=1,inplace=True)
    Best_Car = df7.iloc[-1:]
    return df.loc[Best_Car.index]


In [ ]:
Best_Car('model_class_lexus')

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
66877,1200,2013,lexus,es 350,excellent,6 cylinders,gas,92699.0,clean,automatic,fwd,UNKNOWN,sedan,UNKNOWN,fl


In [ ]:
Best_Car('model_class_jeep wrangler')

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
5653,2000,2015,jeep,wrangler jku,good,UNKNOWN,gas,1.0,clean,other,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,az
